In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
station = Base.classes.station
measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# examine the data, see a sample row
first_row_station = session.query(station).first()
first_row_station.__dict__

In [ ]:
# create inspector object and find the table names
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Print column names and types for the measurement table
meas_columns = inspector.get_columns('measurement')
for column in meas_columns:
    print(column['name'], column['type'])

## Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set, convert string format to date.
most_recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]
most_recent_date = (dt.datetime.strptime(most_recent_date, '%Y-%m-%d')).date()
most_recent_date

In [ ]:
# Calculate the date one year prior to the most recent date in dataset
analysis_start_date = most_recent_date - dt.timedelta(days=365)
analysis_start_date

In [ ]:
# Perform a query to retrieve the data and precipitation scores
prcp_data = session.query(measurement.date, measurement.prcp).\
    filter((measurement.date <= most_recent_date) & (measurement.date >= analysis_start_date)).all()
print(prcp_data[0])
print(prcp_data[0].date)
print(prcp_data[0].prcp)


In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
meas_df = pd.DataFrame(prcp_data)
meas_df = meas_df.set_index('date')
meas_df.head()

In [ ]:
# Sort the DataFram by date and drop rows containing NaN values
meas_df = meas_df.sort_index()
meas_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [ ]:
# Rename column for cleaner DataFrame and for plot legend
meas_df = meas_df.rename(columns = {'prcp': 'Precipitation'})
meas_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
panda_plot = meas_df.plot(figsize = (18,10), color="blue", alpha=1, legend=True, rot=45)

panda_plot.set_title(f'Daily Precipitation {most_recent_date} - {analysis_start_date}', fontsize=18)
panda_plot.set_xlabel('Date', fontsize=14)
panda_plot.set_ylabel('Precipitation in inches', fontsize=14)

# Save plot to image file
plt.savefig('Images/Daily_Precipitation_panda_plot.jpg')

# Display plot
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
meas_df.describe()


## Exploratory Station Analysis

In [ ]:
# Print column names and types for the station table
sta_columns = inspector.get_columns('station')
for column in sta_columns:
    print(column['name'], column['type'])

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(station).count()

print(f'The total number of stations is: {total_stations}')

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
most_active = session.query(measurement.station, func.count(measurement.station), station.name, station.id).\
    order_by(func.count(measurement.station).desc()).\
    group_by(measurement.station).all()

# List the stations and the counts in descending order.
most_active

In [ ]:
# Declare variables to store extracted tuples from list
most_active_station = most_active[0]
print(f'The most active station is {most_active_station[0]} with a measurement count of {most_active_station[1]}')

In [ ]:
count_by_station = []   #gonna create a list of tuples
for row in results:
    #print(row[1])
    measurements = session.query(Measurement.station, func.count(distinct(Measurement.prcp))).filter(Measurement.station == row[1]).all()
    print(measurements)
    count_by_station.append(measurements[0])
 
# count_by_station
station_count_df = pd.DataFrame(count_by_station, columns = ['station', 'measurement_count'])
station_count_sorted_df = station_count_df.sort_values('measurement_count', ascending=False)
station_count_sorted_df

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
# Lowest temperature for most active station is 'USC00516128'
results = session.query(func.min(Measurement.tobs)).filter(Measurement.station == 'USC00516128').all()
print(results[0][0])

In [ ]:
# Highest temperature for most active station 'USC00516128'
results = session.query(func.max(Measurement.tobs)).filter(Measurement.station == 'USC00516128').all()
print(results[0][0])

In [ ]:
# Average temperature from most active station 'USC00516128'
results = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == 'USC00516128').\
    filter(Measurement.tobs != 'None').all()
print(results[0][0])
print(round(results[0][0],6))



In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == 'USC00516128').\
            filter(Measurement.date > '2016-08-23').order_by(Measurement.date.desc()).\
            filter(Measurement.tobs != 'None').all()
results

In [ ]:
active_df = pd.DataFrame(results, columns = ['date', 'tobs'])
#active_df = pd.DataFrame(results, columns = ['tobs'])
active_df

In [ ]:
import matplotlib.pyplot as plt

x_data = active_df['date']
y_data = active_df['tobs']

fig, ax = plt.subplots(1,1)

# the following gets:  ValueError: `bins` must increase monotonically, when an array
# ax.hist(x_data, bins=['2016-08-01','2016-09-01','2016-10-01',
#                          '2016-11-01','2016-12-01','2017-01-01', 
#                          '2017-02-01','2017-03-01','2017-04-01',
#                          '2017-05-01','2017-06-01','2017-07-01',
#                          '2017-08-01','2017-09-01'])
ax.hist(x_data, bins=12, density=True)
#ax[1].hist(y_data, bin)
ax.set_title('Station USC00516128 \n2016-08-23 to 2017-08-23')
#ax.set_xticks(['2016-08-01','2016-11-01','2017-02-01','2017-05-01','2017-08-01'])
ax.plot(x_data, y_data, color='blue', alpha=0.5)
ax.set_xlabel('dates')
ax.set_ylabel('temperature observation data')
fig.tight_layout()
plt.ylim([55,85])
plt.show()

# Close session

In [ ]:
# Close Session
session.close()